# Lib import

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import seaborn as sns

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  
from matplotlib import cm

# Загрузка данных и просмотр их первоначальной структуры

Загружаем датасет с GitHub:

In [ ]:
import zipfile
import os

!wget --no-check-certificate \
    "https://github.com/PolMix/nems_ai/archive/refs/heads/main.zip" \
    -O "/tmp/data.zip"


zip_ref = zipfile.ZipFile('/tmp/data.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-03-31 16:46:22--  https://github.com/PolMix/nems_ai/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/PolMix/nems_ai/zip/refs/heads/main [following]
--2023-03-31 16:46:23--  https://codeload.github.com/PolMix/nems_ai/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/data.zip’

/tmp/data.zip           [      <=>           ]  10.27M  5.85MB/s    in 1.8s    

2023-03-31 16:46:25 (5.85 MB/s) - ‘/tmp/data.zip’ saved [10770546]



In [ ]:
%cd /tmp/nems_ai-main/Data
filename = 'Data_10000nm.txt'
df = pd.read_fwf(filename)

/tmp/nems_ai-main/Data


"Читаем" датасет:

In [ ]:
df.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Eigenfrequency (Hz),Quality factor,Effective mass (kg),Noise (kg^2/s^3),TED (W)
0,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,169003.78980494454+1.5691739530118671i,53851.196510,2.581146e-13,1.490593e-28,9.642600e-13
1,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,338502.68077133945+3.1429555853411406i,53851.012459,2.581529e-13,2.986005e-28,5.677122e-12
2,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,508990.0685003006+4.725920015554869i,53850.897479,2.582148e-13,4.490996e-28,1.441984e-11
3,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,680954.0843095792+6.321728894497595i,53858.216294,2.583005e-13,6.009473e-28,2.286573e-11
4,187.634453,482.065578,150.0,19.450374,161.773060,911.580688,10.599196,512665.9173008662+8.862420709879311i,28923.582737,2.167964e-14,9.751216e-26,6.273790e-12


# Функции для обработки датасета

Функция, реализующая трансформацию датасета 4row x 1col --> 1row x 4col:

In [ ]:
def process_data(df, len_common):
    # num_common - количество стоблцов, одинаковых для всех мод
    data = []
    for index in range(df.shape[0]):
        if index % 4 == 0:
            i = 0
            row = df.iloc[index].values.flatten().tolist()
        else:
            row += df.iloc[index, len_common:].values.flatten().tolist()
        i += 1
        if i == 4:
            data.append(row)
    return data

Функция, реализующая замену индексовых имен колонок на буквенные:

In [ ]:
def name_data_columns(df, data, len_common):
    #len_common - число колонок, общих для всех резонансных мод
    len_differ = df.shape[1] - len_common # число различающихся колонок

    cols_common = list(df.columns)[:len_common]
    cols_differ = list(df.columns)[len_common:]
    print(len_differ, len(cols_differ))

    columns = [] # сюда будем записывать финальный вариант колонок.
    # Прим.: pandas не дает изменять колонки внутри цикла,
    # их можно изменить только единственным присваиванием
    
    # проименовываем общие колонки:
    for col_index in range(0, len_common):
        columns.append(cols_common[col_index])
    
    # проименовываем различающиеся колонки:
    for mode_index in range(1, 5):
        for col_index in range(0, len_differ):
            columns.append(f'M{mode_index} ' + cols_differ[col_index])
    data.columns = columns

    return data

Функция, возвращающая индексы строк (и столбцов, если рассматривать разные моды колебаний), в которых детектирована аномалия $ Im\left(f_0\right) < 0$:

In [ ]:
def return_neg_frequencies_index(df, freq_indices):
    list_i = []
    list_j = []
    for j in freq_indices:
        for i in range(df.shape[0]):
            if '-' in df.iloc[i,j]:
                list_i.append(i)
                list_j.append(j)
    return list_i, list_j

Функция, реализующая удаление описанных выше аномалий:

In [ ]:
def del_neg_frequencies(df, freq_indices):
    lst_i, _ = return_neg_frequencies_index(data, freq_indices)
    lst_i.sort(reverse=True)
    for i in lst_i:
        df.drop(index=i, inplace=True)
    return df

Функция, реализующая удаление мнимой части резонансной частоты для всех элементов датасета:

In [ ]:
def del_im_frequency(df, freq_indices):
    for j in freq_indices:
        for i in range(0, df.shape[0]):
            df.iloc[i, j] = float(str(df.iloc[i, j]).split('+', 1)[0])
    return df

Функция, реализующая округление чисел до `decimals` знаков после запятой (потому что метод конечных элементов иногда, например, вместо числа `200` записывает `200.0000000111`):

In [ ]:
def round_data_float(data, decimals):
    return data.round(decimals=decimals)

Функция, реализующая округление значений до типа `int`, которые должны были быть записаны методом конечных элементов как `int`, но были записаны как `float`:

In [ ]:
def round_data_int(data, cols_indices_to_round):
    for col_index in cols_indices_to_round:
        data.iloc[:, col_index] = data.iloc[:, col_index].astype(int)

# Обработка датасета

Преобразование 4row x 1col --> 1row x 4col:

In [ ]:
data = pd.DataFrame(process_data(df, len_common=7))
data.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,169003.78980494454+1.5691739530118671i,53851.196510,2.581146e-13,...,508990.0685003006+4.725920015554869i,53850.897479,2.582148e-13,4.490996e-28,1.441984e-11,680954.0843095792+6.321728894497595i,53858.216294,2.583005e-13,6.009473e-28,2.286573e-11
1,187.634453,482.065578,150.0,19.450374,161.773060,911.580688,10.599196,512665.9173008662+8.862420709879311i,28923.582737,2.167964e-14,...,1616819.3527052735+27.955164052453224i,28918.080210,2.199621e-14,3.120787e-25,1.475224e-11,2243524.988027999+38.800934216497126i,28910.708380,2.212375e-14,4.356675e-25,1.758884e-11
2,638.204045,242.947138,100.0,24.144951,3.181935,455.886263,15.074379,221111.86993594936+7.584434554678179i,14576.687843,2.871363e-14,...,663629.6857651912+22.763180780703948i,14576.822373,2.871555e-14,5.640215e-28,7.054148e-13,885178.9825786535+30.362487466306842i,14576.852169,2.871377e-14,7.522689e-28,1.155275e-12
3,910.105786,675.544397,200.0,43.676386,107.099808,914.705274,8.740595,124047.99125075326+1.5301995367248011i,40533.273038,2.224241e-13,...,373133.78371345735+4.60262905893176i,40534.852900,2.224716e-13,1.462375e-26,7.550186e-12,498662.98268608114+6.148884334963796i,40549.061872,2.225221e-13,1.954104e-26,1.220765e-11
4,665.998773,627.258997,200.0,26.364768,0.970622,659.057305,0.787551,201641.65294438173+2.6788698793321255i,37635.581799,1.412254e-13,...,606842.5081818213+8.062091029389682i,37635.552983,1.412640e-13,2.752680e-28,4.751424e-12,811354.6500383065+10.778025941444263i,37639.297514,1.412993e-13,3.680917e-28,1.052563e-11


Форматирование названий колонок:

In [ ]:
data = name_data_columns(df, data, len_common=7)
data.head()

5 5


,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
0,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,169003.78980494454+1.5691739530118671i,53851.196510,2.581146e-13,...,508990.0685003006+4.725920015554869i,53850.897479,2.582148e-13,4.490996e-28,1.441984e-11,680954.0843095792+6.321728894497595i,53858.216294,2.583005e-13,6.009473e-28,2.286573e-11
1,187.634453,482.065578,150.0,19.450374,161.773060,911.580688,10.599196,512665.9173008662+8.862420709879311i,28923.582737,2.167964e-14,...,1616819.3527052735+27.955164052453224i,28918.080210,2.199621e-14,3.120787e-25,1.475224e-11,2243524.988027999+38.800934216497126i,28910.708380,2.212375e-14,4.356675e-25,1.758884e-11
2,638.204045,242.947138,100.0,24.144951,3.181935,455.886263,15.074379,221111.86993594936+7.584434554678179i,14576.687843,2.871363e-14,...,663629.6857651912+22.763180780703948i,14576.822373,2.871555e-14,5.640215e-28,7.054148e-13,885178.9825786535+30.362487466306842i,14576.852169,2.871377e-14,7.522689e-28,1.155275e-12
3,910.105786,675.544397,200.0,43.676386,107.099808,914.705274,8.740595,124047.99125075326+1.5301995367248011i,40533.273038,2.224241e-13,...,373133.78371345735+4.60262905893176i,40534.852900,2.224716e-13,1.462375e-26,7.550186e-12,498662.98268608114+6.148884334963796i,40549.061872,2.225221e-13,1.954104e-26,1.220765e-11
4,665.998773,627.258997,200.0,26.364768,0.970622,659.057305,0.787551,201641.65294438173+2.6788698793321255i,37635.581799,1.412254e-13,...,606842.5081818213+8.062091029389682i,37635.552983,1.412640e-13,2.752680e-28,4.751424e-12,811354.6500383065+10.778025941444263i,37639.297514,1.412993e-13,3.680917e-28,1.052563e-11


Вывод информации об аномалиях мнимой части резонансной частоты, их удаление:

In [ ]:
frequency_indices = [7, 12, 17, 22] # индексы колонок, в которых содержатся значения резонансных частот

# Детектирование аномалий:
lst_i, lst_j = return_neg_frequencies_index(data, frequency_indices)
print("Row and column indices of detected anomalies of imaginary part of resonant frequency:")
print(lst_i, lst_j)
print(f"The length of the dataset is {data.shape[0]}")

# Удаление аномалий:
data = del_neg_frequencies(data, frequency_indices)
lst_i, lst_j = return_neg_frequencies_index(data, frequency_indices)
print(f"Anomalies have been deleted, the length of the dataset is {data.shape[0]}")

Row and column indices of detected anomalies of imaginary part of resonant frequency:
[161, 2947, 248, 332, 754, 911, 918, 934, 995, 1018, 1382, 1495, 1506, 1537, 1558, 1665, 1774, 1831, 1840, 1909, 1950, 1996, 2002, 2007, 2020, 2026, 2086, 2116, 2147, 2167, 2203, 2248, 2322, 2375, 2390, 2443, 2504, 2532, 2595, 2629, 2657, 2664, 2689, 2710, 2742, 2761, 2765, 2792, 2823, 2878, 2938, 2943, 2959, 2960, 3030, 3069, 3372] [17, 17, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22]
The length of the dataset is 9296
Anomalies have been deleted, the length of the dataset is 9239


Удаление мнимой части всех элементов датасета:

In [ ]:
data = del_im_frequency(data, [7, 12, 17, 22])
data.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
0,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,169003.789805,53851.196510,2.581146e-13,...,508990.0685,53850.897479,2.582148e-13,4.490996e-28,1.441984e-11,680954.08431,53858.216294,2.583005e-13,6.009473e-28,2.286573e-11
1,187.634453,482.065578,150.0,19.450374,161.773060,911.580688,10.599196,512665.917301,28923.582737,2.167964e-14,...,1616819.352705,28918.080210,2.199621e-14,3.120787e-25,1.475224e-11,2243524.988028,28910.708380,2.212375e-14,4.356675e-25,1.758884e-11
2,638.204045,242.947138,100.0,24.144951,3.181935,455.886263,15.074379,221111.869936,14576.687843,2.871363e-14,...,663629.685765,14576.822373,2.871555e-14,5.640215e-28,7.054148e-13,885178.982579,14576.852169,2.871377e-14,7.522689e-28,1.155275e-12
3,910.105786,675.544397,200.0,43.676386,107.099808,914.705274,8.740595,124047.991251,40533.273038,2.224241e-13,...,373133.783713,40534.852900,2.224716e-13,1.462375e-26,7.550186e-12,498662.982686,40549.061872,2.225221e-13,1.954104e-26,1.220765e-11
4,665.998773,627.258997,200.0,26.364768,0.970622,659.057305,0.787551,201641.652944,37635.581799,1.412254e-13,...,606842.508182,37635.552983,1.412640e-13,2.752680e-28,4.751424e-12,811354.650038,37639.297514,1.412993e-13,3.680917e-28,1.052563e-11


Дополнительно удалим строку с индексом `2`, потому что в ячейке `[2, 'M1 TED (W)']` содержится аномалия в данных (неопознанное вторжение символа `s` в `float`-значение) (поймано на графиках распределений):

In [ ]:
#data = data.drop(2)
#print(df.iloc[0:3, 17])

In [ ]:
data.describe()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Quality factor,M1 Effective mass (kg),M1 Noise (kg^2/s^3),...,M2 Noise (kg^2/s^3),M2 TED (W),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
count,9239.000000,9239.000000,9239.000000,9239.000000,9239.000000,9239.000000,9239.000000,9239.000000,9.239000e+03,9.239000e+03,...,9.239000e+03,9.239000e+03,9239.000000,9.239000e+03,9.239000e+03,9.239000e+03,9.239000e+03,9.239000e+03,9.239000e+03,9.239000e+03
mean,307.707557,598.882503,149.886351,30.182628,42.429805,700.196237,20.068474,35884.109802,4.820010e-14,2.547359e-24,...,7.431695e-24,1.626558e-11,35780.959759,4.844185e-14,1.226906e-23,2.143503e-11,3.620661e+04,4.813487e-14,5.462583e-23,1.332063e-11
std,284.900618,295.877191,40.898626,11.550921,61.318219,173.303713,11.525626,17709.663642,5.603569e-14,4.695030e-23,...,1.296224e-22,1.663991e-11,17666.469957,5.604834e-14,1.861721e-22,2.094353e-11,4.770728e+04,5.609109e-14,1.227450e-21,1.691487e-11
min,5.020619,50.035027,100.000000,10.000042,0.001210,400.208659,0.001029,3002.080573,9.769558e-17,2.487767e-31,...,4.979344e-31,5.294733e-15,3002.085540,1.015906e-16,7.478469e-31,1.575068e-14,2.198490e+02,1.008555e-16,1.001906e-30,6.913949e-15
25%,125.728924,348.759737,100.000000,20.157978,5.448914,549.318852,10.046357,20916.429103,1.071475e-14,5.116261e-27,...,8.643930e-27,3.229421e-12,20891.973944,1.091802e-14,1.216264e-26,5.126080e-12,2.047662e+04,1.078136e-14,4.774224e-27,2.730294e-12
50%,202.178233,596.540435,150.000000,30.504323,12.585116,699.963680,20.166376,35768.226592,2.903045e-14,1.956135e-26,...,3.176068e-26,1.086471e-11,35649.209706,2.931599e-14,4.462187e-26,1.527109e-11,3.513432e+04,2.884695e-14,1.396058e-26,8.111206e-12
75%,381.694057,850.149493,200.000000,40.215650,46.342718,850.714439,30.050402,50950.278994,6.247186e-14,6.841450e-26,...,1.182911e-25,2.436770e-11,50818.101468,6.292494e-14,1.733737e-25,3.143106e-11,5.055219e+04,6.228501e-14,7.358594e-26,1.695476e-11
max,1189.333737,1149.817461,200.000000,49.999601,249.897882,999.832165,39.999497,68984.076194,4.233912e-13,3.013697e-21,...,7.460981e-21,1.046226e-10,137776.418944,4.234602e-13,9.366819e-21,1.531540e-10,3.766407e+06,4.235220e-13,6.437679e-20,2.438053e-10


Дополнительно поймаем аномалию, связанную с сильно заниженным значением термоупругих потерь четвертой моды (поймано на графиках распределений). Одновременно столбец резонансных частот конвертируется в тип `float`.

In [ ]:
def catch_parameter_overflow(df, param_name, lower_limit, upper_limit):
    mask = (df[param_name].astype(float) > lower_limit) & (df[param_name].astype(float) < upper_limit)
    return df[mask]

def catch_overflow(df, param_limits):

    # Это кусок кода на случай, если на рез. частоту ограничений не накладывается, 
    #   но в любом случае надо конвертировать данные в тип float
    if 'Eigenfrequency (Hz)' not in param_limits.keys():
        for mode_number in range(1, 5):
            df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'] = df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'].astype(float)

    for param in param_limits.keys():
        for mode_number in range(1, 5):

            # Преобразуем тип резонансной частоты в тип float
            if param == 'Eigenfrequency (Hz)':
                df.loc[:, f'M{mode_number} ' + param] = df.loc[:, f'M{mode_number} ' + param].astype(float)

            # Последовательно накладываем маски ограничений
            df = catch_parameter_overflow(df, param_name=f'M{mode_number} ' + param,
                                          lower_limit=param_limits[param][0],
                                          upper_limit=param_limits[param][1])
    return df

In [ ]:
param_limits = {#'Eigenfrequency (Hz)': [0, 3e7], 
                'Quality factor': [0, 80000], 
                'TED (W)': [1e-20, 1]}

print('Dataset length before anomaly check is {length}'.format(length=data.shape[0]))
data_catched = catch_overflow(data, param_limits)
print('Dataset length after anomaly check is {length}'.format(length=data_catched.shape[0]))
data_catched.describe()

Dataset length before anomaly check is 9239
Dataset length after anomaly check is 9224


,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
count,9224.000000,9224.000000,9224.000000,9224.000000,9224.000000,9224.000000,9224.000000,9.224000e+03,9224.000000,9.224000e+03,...,9.224000e+03,9224.000000,9.224000e+03,9.224000e+03,9.224000e+03,9.224000e+03,9224.000000,9.224000e+03,9.224000e+03,9.224000e+03
mean,308.177680,598.425928,149.897008,30.183897,42.357168,700.152338,20.079607,3.258400e+06,35859.694288,4.827316e-14,...,1.237381e+07,35752.458215,4.851497e-14,1.188751e-23,2.145791e-11,1.514273e+07,35455.190158,4.820884e-14,5.414631e-23,1.332879e-11
std,284.893227,295.811823,40.905402,11.551165,61.279931,173.313397,11.526893,1.148645e+07,17709.034795,5.605190e-14,...,4.984188e+07,17638.080018,5.606450e-14,1.854906e-22,2.095198e-11,6.648890e+07,17733.706677,5.610662e-14,1.227734e-21,1.692456e-11
min,5.020619,50.035027,100.000000,10.000042,0.001210,400.208659,0.001029,4.268006e+04,3002.080573,9.769558e-17,...,1.288968e+05,3002.085540,1.015906e-16,7.478469e-31,1.575068e-14,1.272115e+05,219.848988,1.008555e-16,1.001906e-30,6.913949e-15
25%,126.585298,348.554808,100.000000,20.160924,5.448936,549.146417,10.048107,5.133915e+05,20898.395022,1.078578e-14,...,1.245952e+06,20840.718129,1.098722e-14,1.212216e-26,5.133675e-12,5.511361e+05,20464.377674,1.086094e-14,4.763635e-27,2.719678e-12
50%,202.378990,596.291222,150.000000,30.506147,12.569990,699.983174,20.181168,1.299929e+06,35738.414361,2.911695e-14,...,2.840347e+06,35597.833250,2.937037e-14,4.446493e-26,1.532070e-11,8.485523e+05,35101.970191,2.892037e-14,1.394083e-26,8.119326e-12
75%,382.258223,849.719460,200.000000,40.213606,46.233018,850.577760,30.062994,2.135130e+06,50931.335697,6.256295e-14,...,5.020868e+06,50813.754538,6.295261e-14,1.721616e-25,3.144812e-11,1.700453e+06,50503.293805,6.236941e-14,7.275640e-26,1.696864e-11
max,1189.333737,1149.817461,200.000000,49.999601,249.897882,999.832165,39.999497,2.532571e+08,68984.076194,4.233912e-13,...,8.125858e+08,68981.163476,4.234602e-13,9.366819e-21,1.531540e-10,9.722923e+08,76639.661104,4.235220e-13,6.437679e-20,2.438053e-10


Дополнительно: еще раз изменим тип всех данных на `float`:

In [ ]:
for param in list(data_catched.columns)[:7]:
    data_catched.loc[:, param] = data_catched.loc[:, param].astype(float)

for row_index in range(data_catched.shape[0]):
    if data_catched.loc[data_catched.index[row_index], 'Gate voltage (V)'] == '5.630590021610321E':
        print(row_index)

for param in list(data_catched.columns)[7:]:
    data_catched.loc[:, param] = data_catched.loc[:, param].astype(float)

# Добавление вклада термоупругих потерь

Подмешаем к добротности в датасете добротность согласно приближению Roukes:

In [ ]:
# Fundamental properties:
rho = [3100, 2700] # density of matter
E = [250e9, 70e9] # Young's modulus
k = [20, 237] # Thermal conductivity constant
alpha = [2.3e-6, 23.1e-6] # Thermal expansion coefficient

# Calculated properties:
Cp = [700, 904] # heat capacity at constant pressure

In [ ]:
def calculate_Q_TED(df):
    param_history = {'Cp_av': [], 'E_av': [], 'rho_av': [], 'k_av': [], 'alpha_av': [], 'xi_av': [], 'chi': [], 'Q_ted': []}
    for row_index in range(0, df.shape[0]):
        # Calculating average properties of beam:
        Cp_av = (Cp[0] * df.iloc[row_index, 2] + Cp[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        E_av = (E[0] * df.iloc[row_index, 2] + E[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        rho_av = (rho[0] * df.iloc[row_index, 2] + rho[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        k_av = (k[0] * df.iloc[row_index, 2] + k[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        alpha_av = (alpha[0] * df.iloc[row_index, 2] + alpha[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        xi_av = k_av / (rho_av * Cp_av)
        param_history['Cp_av'].append(Cp_av)
        param_history['E_av'].append(E_av)
        param_history['rho_av'].append(rho_av)
        param_history['k_av'].append(k_av)
        param_history['alpha_av'].append(alpha_av)
        param_history['xi_av'].append(xi_av)

        for mode_number in range(1, 5):
            # Calculating TED quality factor:
            chi = df.iloc[row_index, 1] * 1e-9 * np.sqrt(2 * 3.1415 * df.loc[df.index[row_index], f'M{mode_number} Eigenfrequency (Hz)'] / (2 * xi_av))
            delta_ted = (E_av * alpha_av **2 * df.iloc[row_index, 4]) / (Cp_av * rho_av) * (6/(chi**2) - 6 * (np.sinh(chi) + np.sin(chi))/(chi**3 * (np.cosh(chi) + np.cos(chi))))
            df.loc[df.index[row_index], f'M{mode_number} Quality factor'] = 1/(1/df.loc[df.index[row_index], f'M{mode_number} Quality factor'] + delta_ted)

            param_history['chi'].append(chi)
            param_history['Q_ted'].append(1/delta_ted)
        
    return df, param_history

In [ ]:
print(data_catched.shape[0])
data_ted, history = calculate_Q_TED(data_catched)

9224


In [ ]:
sorted_history = sorted(history['Q_ted'])
print(sorted_history)

[1515.7020416733417, 1579.8174961485242, 1590.7997991070638, 1591.2937383503859, 1592.5663616958689, 1604.1404094829063, 1606.010428896227, 1656.7448408813473, 1712.3014747254092, 1738.4183052839296, 1767.7724491026217, 1795.418361887757, 1814.1105140578206, 1832.5469686363265, 1852.4194923800928, 1922.1088501387324, 1950.2839128619544, 1954.347172107683, 1961.5742138524467, 1961.9264078037174, 2008.809233151855, 2091.348673897435, 2104.95086309399, 2163.956791260071, 2201.1268533565453, 2205.808052405887, 2250.365634480105, 2290.9686713517863, 2335.570028989186, 2369.5118334083377, 2388.9270897038928, 2412.0415787960624, 2419.3098284700436, 2424.7666292237113, 2444.8472244737327, 2457.3532420514102, 2480.453868233955, 2487.130627470288, 2499.6072977522663, 2501.3852196633293, 2504.086550312581, 2520.004390179612, 2539.9322457838643, 2567.4668228019955, 2580.5014161710315, 2591.329333175508, 2607.7069233514694, 2609.4588446113266, 2668.475771368362, 2670.5269929400033, 2672.74164432024

In [ ]:
data_ted.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
0,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,169003.789805,53781.219971,2.581146e-13,...,5.089901e+05,53640.722509,2.582148e-13,4.490996e-28,1.441984e-11,6.809541e+05,53577.353352,2.583005e-13,6.009473e-28,2.286573e-11
1,187.634453,482.065578,150.0,19.450374,161.773060,911.580688,10.599196,512665.917301,27829.840781,2.167964e-14,...,1.616819e+06,25729.966624,2.199621e-14,3.120787e-25,1.475224e-11,2.243525e+06,24670.596346,2.212375e-14,4.356675e-25,1.758884e-11
2,638.204045,242.947138,100.0,24.144951,3.181935,455.886263,15.074379,221111.869936,14575.916394,2.871363e-14,...,6.636297e+05,14574.507206,2.871555e-14,5.640215e-28,7.054148e-13,8.851790e+05,14573.764249,2.871377e-14,7.522689e-28,1.155275e-12
3,910.105786,675.544397,200.0,43.676386,107.099808,914.705274,8.740595,124047.991251,39709.542285,2.224241e-13,...,3.731338e+05,38154.086830,2.224716e-13,1.462375e-26,7.550186e-12,4.986630e+05,37426.945624,2.225221e-13,1.954104e-26,1.220765e-11
4,665.998773,627.258997,200.0,26.364768,0.970622,659.057305,0.787551,201641.652944,37627.851275,1.412254e-13,...,6.068425e+05,37612.298667,1.412640e-13,2.752680e-28,4.751424e-12,8.113547e+05,37608.207889,1.412993e-13,3.680917e-28,1.052563e-11


# Сохраним датасет на диск:

In [ ]:
data_ted.to_csv('Data_Processed_10000nm.csv')